# Rules

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from itertools import combinations

import pdpexplorer
from pdpexplorer.pdp import partial_dependence

from imodels import GreedyRuleListClassifier, SlipperClassifier, BoostedRulesClassifier, OneRClassifier

In [ ]:
df = pd.read_csv('bike-sharing.csv').drop(columns=['yr'])

In [ ]:
X = df.drop(columns=['label', 'prediction'])

In [ ]:
y = (df['label'] >= df['label'].median()).to_numpy().astype(int)

In [ ]:
feature_names = [col for col in X.columns]

In [ ]:
# m = GreedyRuleListClassifier(max_depth=5)
# m.fit(X.drop(columns=['hr']), y=y, feature_names=features)

In [ ]:
# m.rules_

In [ ]:
# m

In [ ]:
# m._print_list()

In [ ]:
# def pred(df):
#     y = []
    
#     for index, row in df.iterrows():
#         if row["hr"] >= 7.0:
#             y.append(1)
#         elif row["hum"] >= 0.58:
#             y.append(0)
#         else:
#             y.append(0)
        
#     return np.array(y)

In [ ]:
# pred(df)

In [ ]:
# m.predict(df)

In [ ]:
# np.all(pred(df) == m.predict(df))

In [ ]:
# df.shape

In [ ]:
# df[df['hr'] >= 7.0].shape

In [ ]:
# df[df['hr'] < 7.0].shape

In [ ]:
# df[(df['hr'] < 7.0) & (df['hum'] < 0.58)].shape

In [ ]:
# m = SlipperClassifier()
# m.fit(X, y=y, feature_names=features);

In [ ]:
# print(m)

In [ ]:
# m = BoostedRulesClassifier()
# m.fit(X, y=y, feature_names=features);

In [ ]:
# print(m)

In [ ]:
# m = OneRClassifier()
# m.fit(X, y=y, feature_names=list(features));

In [ ]:
# m

In [ ]:
# tree.plot_tree(clf, feature_names=feature_names)

In [ ]:
def describe_cluster(X, y, feature_names):
    # This code is adapted from 
    # https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
    clf = DecisionTreeClassifier(max_depth=3, ccp_alpha=0.01)
    clf.fit(X, y)

    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    threshold = clf.tree_.threshold
    feature = clf.tree_.feature
    value = clf.tree_.value
    label = np.argmax(clf.tree_.value, axis=2).ravel()

    stack = [(0, [])]

    rules = []

    while len(stack) > 0:
        node_id, path = stack.pop()
        is_split_node = children_left[node_id] != children_right[node_id]

        if is_split_node:
            left_condition = {
                'feature': feature_names[feature[node_id]],
                'sign': '<=',
                'threshold': threshold[node_id]
            }
            stack.append((children_left[node_id], path + [left_condition]))

            right_condition = {
                'feature': feature_names[feature[node_id]],
                'sign': '>',
                'threshold': threshold[node_id]
            }
            stack.append((children_right[node_id], path + [right_condition]))
            
        elif label[node_id] == 1:
            num_instances = value[node_id].sum()
            num_correct = value[node_id][0,1]

            rule = {
                'conditions': path,
                'num_instances': num_instances,
                'num_correct': num_correct,
                'accuracy': num_correct / num_instances,
            }

            rules.append(rule)
            
    return rules
        
print(describe_cluster(X, y, feature_names))